In [1]:
import pandas as pd
url = 'C:/python_src2/data/San-Francisco-Crime.csv'
df = pd.read_csv(url)
df.shape

(878049, 9)

In [2]:
df.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


In [4]:
# # Domain 지식   컬럼정보
# Dates: 범죄 사건이 발생한 날짜와 시간 (타임스탬프)

# Category: 범죄 사건의 유형 ( 예측해야 할 목표 변수)

# Descript: 범죄 사건에 대한 상세한 설명

# DayOfWeek: 요일

# PdDistrict: 경찰서 관할 구역명

# Resolution: 사건이 어떻게 해결되었는지에 대한 정보

# Address: 범죄 사건이 발생한 대략적인 주소

# X: 경도 (Longitude)

# Y: 위도 (Latitude)

In [24]:
vc = df.Category.value_counts()
keep = vc[vc > 6].index
df_filtered = df[df.Category.isin(keep)]

In [25]:
# 데이터 셈플링
from sklearn.model_selection import train_test_split
X_ = df_filtered.iloc[:,2:]
y_ = df_filtered.Category
X,_,y,_ = train_test_split(X_,y_,stratify=y_, train_size=0.2,random_state=50)
X.shape, y.shape

((175608, 7), (175608,))

In [26]:
# # 8 : 2
from sklearn.svm import SVC
from sklearn.metrics import classification_report
x_train,x_test,y_train,y_test = train_test_split(X,y, stratify=y ,test_size=0.2, random_state=42)

# # svm = SVC()
# # svm.fit(x_train,y_train)
# # y_pred = svm.predict(x_test)
# # print( classification_report(y_test, y_pred) )